# chapter 9. Recommendation System

## Latent Factor Filtering

### 잠재 요인 협업 필터링의 이해

**잠재 요인 협업 필터링**은 **사용자-아이템 평점 매트리스 속에 숨어 있는 잠재 요인을 추출해 추천 예측을 할 수 있게 하는 기법**이다. <br>
대규모 다차원 행렬을 SVD와 같은 **차원 감소 기법으로 분해하는 과정에서 잠재 요인을 추출**하는데, 이한 기법을 **행렬 분해(Matrix Factorization)** 이라고 한다. <br>

다차원 희소 행렬인 **사용자-아이템 행렬 데이터**를 저차원 밀집 행렬의 **사용자-잠재 요인 행렬**과 **아이템-잠재 요인 행렬의 전치 행렬(즉, 잠재 요인-아이템 행렬)** 로 **분해**할 수 있으며, 이렇게 **분해된 두 행렬의 내적을 통해 새로운 예측 사용자-아이템 평점 행렬 데이터를 만들어서 사용자가 아직 평점을 부여하지 않은 아이템에 대한 예측 평점을 생성하는 것**이 잠재 요인 협력 필터링 알고리즘의 기반이다. <br>

### 행렬 분해의 이해 

행렬 분해는 다차원 매트릭스를 저차원 매트릭스로 분해하는 기법으로서 대표적으로 SVD, NMF 등이 있다. <br>

### 확률적 경사 하강법을 이용한 행렬 분해 

확률적 경사 하강법을 이용한 행렬 분해 방법은 **P와 Q 행렬로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 P와 Q를 유추해 내는 것**이다. 

1. P와 Q를 임의의 값을 가진 행렬로 설정한다.
2. P와 Q.T 값을 곱해 예측 R 행렬을 계산하고 예측 R 행렬과 실제 R 행렬에 해당하는 오류 값을 계산한다.
3. 이 오류 값을 최소화할 수 있도록 P와 Q 행렬을 적절한 값으로 업데이트한다.
4. 만족할만한 오류 값을 가질 때까지 2, 3번 작업을 반복하면서 P와 Q 값을 업데이트해 근사화한다.

In [3]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재 요인 차원 K는 3으로 설정한다. 
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN], 
              [np.NaN, 5, np.NaN, 3, 1], 
              [np.NaN, np.NaN, 3, 4, 4], 
              [5, 2, 1, 2, np.NaN]])

num_users, num_items = R.shape
K = 3

# P와 Q 행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력한다. 
np.random.seed(1)
P = np.random.normal(scale = 1./K, size = (num_users, K))
Q = np.random.normal(scale = 1./K, size = (num_items, K))

In [12]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0

    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 Null이 아닌 값의 위치 인덱스를 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return mse

In [13]:
# R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장한다. 
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트 
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값을 구한다. 
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식을 적용
        P[i, :] = P[i, :] + learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])
        Q[j, :] = Q[j, :] + learning_rate * (eij * P[i, :] - r_lambda * Q[j, :])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0:
        print('iteration step :', step, 'rmse : ', rmse)

iteration step : 0 rmse :  10.00878096548348
iteration step : 50 rmse :  0.20225300999696663
iteration step : 100 rmse :  0.028133574950373216
iteration step : 150 rmse :  0.00778830199770756
iteration step : 200 rmse :  0.0030323165594135725
iteration step : 250 rmse :  0.0014953084965356213
iteration step : 300 rmse :  0.000885076527371714
iteration step : 350 rmse :  0.0006116872905032688
iteration step : 400 rmse :  0.0004785027694735831
iteration step : 450 rmse :  0.00040876454402848875
iteration step : 500 rmse :  0.00036966305483060756
iteration step : 550 rmse :  0.0003462508165294459
iteration step : 600 rmse :  0.00033132240217969715
iteration step : 650 rmse :  0.00032121094704807077
iteration step : 700 rmse :  0.00031395330548720704
iteration step : 750 rmse :  0.00030845024146149695
iteration step : 800 rmse :  0.0003040624558004854
iteration step : 850 rmse :  0.0003004062120261783
iteration step : 900 rmse :  0.0002972449202160699
iteration step : 950 rmse :  0.0002944

In [14]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬 :', np.round(pred_matrix, 3))

예측 행렬 : [[3.99  0.543 1.428 2.003 1.879]
 [6.588 4.978 0.9   2.98  1.003]
 [7.348 0.566 2.987 3.976 3.986]
 [4.967 2.004 1.008 2.02  1.214]]
